# Process All Spectra

(Can get around memory limit in Jupyter Notebook)

In [1]:
print('ping')

ping


In [2]:
'''
Code for reading in 1-d DEIMOS spectra reduced by the spec2d code:
http://deep.ps.uci.edu/spec2d/. Can process spectra from a single target
or from a list of targets sequentially.

This code:
1) Reads in 1-d DEIMOS spectra .fits files and concatenates spectra from the
red and blue CCDs.
2) Interpolates spectra onto the standard wavelength template for DEIMOS.
3) Outputs all object spectra in a .npz file.
'''

from __future__ import absolute_import, division, print_function
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
import multiprocessing
import utils

# # # Settings # # #


# DEIMOS Directory
DEIMOSDir = '/global/scratch/nathan_sandford/DEIMOS/'
# D-Payne Directory
D_PayneDir = utils.D_PayneDir
# Output directory for processed spectra
OutputDir = D_PayneDir + 'spectra/obs_spectra/'

'''
List of targets to process.
Note that 'm92' is missing and 'pegdig' includes data from two nights.
''' # 'm2', 'm13', 'm15','m71', 'ngc2419', 'ngc7006', 'pegdig', 'pegdig24', already done
targets = ['pegdig26', 'wlm']

'''
Which method of fitting the DEIMOS spectrum do you want to use:
'Horne' or 'Bxspf'
'''
method = 'Bxspf'

# Restore Wavelength Template
print('Restoring Wavelength Template')
wavelength_template = utils.load_wavelength_array()

# Calculate matrix of distances between wavelengths
print('Calculating matrix of distances between wavelengths...')
wavelength_diff_matrix \
    = wavelength_template[:, np.newaxis] - wavelength_template

# Restore DEIMOS continuum pixels
print('Restoring DEIMOS continuum pixels')
cont_reg = utils.load_deimos_cont_pixels()

# Restore telluric mask from Kirby+ 2008
print('Restoring telluric mask from Kirby+ 2008...')
kirby_2008_telluric = utils.get_spectral_mask_dict(name='kirby_2008_telluric')
mask = utils.generate_mask_from_dict(**kirby_2008_telluric)

# Restore spectral template of typical RGB star
print('Restoring spectral template...')
temp = np.load(D_PayneDir + '/other_data/typical_RGB_spectra.npz')
template_spec = temp['spec'][(wavelength_template > 8450) &
                             (wavelength_template < 8700)]
temp.close()

# Prepping cross-correlation function
print('Prepping cross-correlation calculation...')
wavelength_CaIItriplet = wavelength_template[(wavelength_template > 8450) &
                                             (wavelength_template < 8700)]
dv_grid = np.linspace(-500, 250, 750)
template_grid = np.empty((len(dv_grid), len(wavelength_CaIItriplet)))
for i, dv in enumerate(dv_grid):
    template_grid[i] = utils.doppler_shift(wavelength=wavelength_CaIItriplet,
                                           flux=template_spec, dv=dv)

def get_deimos_spectra(Obj, method, InputDir=None):
    '''
    Read in and concatenate red and blue spectra
    '''
    ObjHDUL = fits.open(InputDir + Obj)

    try:
        waveB = ObjHDUL[method+'-B'].data['LAMBDA'][0]
        waveR = ObjHDUL[method+'-R'].data['LAMBDA'][0]
        wave = np.concatenate((waveB, waveR))
    
        specB = ObjHDUL[method+'-B'].data['SPEC'][0]
        specR = ObjHDUL[method+'-R'].data['SPEC'][0]
        spec = np.concatenate((specB, specR))
    
        ivarB = ObjHDUL[method+'-B'].data['IVAR'][0]
        ivarR = ObjHDUL[method+'-R'].data['IVAR'][0]
        ivarB = 1e-16 * np.ones(len(ivarB))  # completely ignore blue channel
        ivar = np.concatenate((ivarB, ivarR))
        ivar[ivar == 0] = 1e-16  # Avoid np.inf in spec_err
    except KeyError:
        print('%s Missing %s spectra' % (Obj, method))
        wave, spec, ivar = np.zeros((3,len(wavelength_template)))
        

    RA = ObjHDUL[method+'-B'].header['RA_OBJ']
    Dec = ObjHDUL[method+'-B'].header['DEC_OBJ']
    try:
        RA_Dec = SkyCoord(RA+' '+Dec, unit=(u.hourangle, u.deg))
    except ValueError:
        print('Missing RA or Dec')
        RA_Dec = SkyCoord('00:00:01 00:00:01', unit=(u.hourangle, u.deg))
    return (wave, spec, ivar, RA_Dec)


def interpolate_deimos_spectra(wave, spec, spec_err):
    '''
    Interpolates a DEIMOS spectrum onto the default wavelength grid
    '''
    if len(wave) != 16250:
        spec = np.interp(wavelength_template, wave, spec)
        spec_err = np.interp(wavelength_template, wave, spec_err)
        wave = np.copy(wavelength_template)
    return(wave, spec, spec_err)


def fast_RV(Obj, spec, spec_err, plot=False):
    '''
    Quick radial velocity determination by cross-correlating observed spectrum
    with a template spectrum in the region around the Ca II triplet.
    '''

    # Consider only region around Ca II triplet
    temp_spec = spec[(wavelength_template > 8450) &
                     (wavelength_template < 8700)]
    temp_spec_err = spec_err[(wavelength_template > 8450) &
                             (wavelength_template < 8700)]

    # Cross-Correlate
    num = np.sum(template_grid * temp_spec / temp_spec_err, axis=1)
    den = np.sum(template_grid * template_grid / temp_spec_err, axis=1)
    xcorr = num / den

    # Naively take maximum of CCF
    dv = dv_grid[np.argmax(xcorr)]

    # Plot output
    if plot:
        plt.plot(dv_grid, xcorr)
        plt.xlabel('RV (km/s)')
        plt.ylabel('Cross-Corellation')
        plt.title('%s' % Obj)
        plt.show()

    return(dv)


def process_deimos_spectra(i):
    '''
    Processes all spectra in InputList
    '''
    Obj = ObjList[i]
    ObjNumber = Obj[14:-5]
    print('Processing spectra for object: %s' % Obj)

    print('Restoring spectra #%s' % ObjNumber)
    wave_temp, spec_temp, ivar_temp, RA_Dec = \
        get_deimos_spectra(Obj=Obj, method=method, InputDir=InputDir)

    print('Interpolating spectra #%s' % ObjNumber)
    wavelength, spec, spec_err = \
        interpolate_deimos_spectra(wave=wave_temp,
                                   spec=spec_temp,
                                   spec_err=(ivar_temp**-1))

    print('Applying telluric mask for spectra %s' % ObjNumber)
    spec_err[mask] = 1e16

    print('Finding radial velocity from CCF for spectra #%s' % ObjNumber)
    dv = fast_RV(Obj, spec, spec_err, plot=False)
    print('%s has Radial Velocity = %.0f' % (ObjNumber, dv))
    print('Shifting spectra %s to rest frame' % ObjNumber)
    spec = utils.doppler_shift(wavelength=wavelength, flux=spec, dv=-dv)

    print('Normalizing spectra %s' % ObjNumber)
    cont_spec = \
        utils.get_deimos_continuum(spec, spec_err=spec_err,
                                   wavelength=wavelength,
                                   cont_pixels=cont_reg,
                                   wavelength_diff_matrix=wavelength_diff_matrix)
    norm_spec = spec / cont_spec

    # Handle regions where continuum is zero
    spec_err[np.isnan(norm_spec)] = 1e16
    norm_spec[np.isnan(norm_spec)] = 0

    return(ObjNumber, wavelength, spec, norm_spec, spec_err, dv, RA_Dec)

for target in targets:
    print('Beginning reduction for %s' % target)
    
    # Input List of DEIMOS spectra to be processed for fitting
    InputDir = DEIMOSDir + 'U112_reduced/'+target+'/'
    InputList = InputDir + 'spec1d.'+target+'msk.txt'
    # File containing processed spectra
    OutputFile = target+'_'+method+'.npz'
    
    # Extract object names from input list
    print('Restoring list of %s objects' % target)
    temp = open(InputList, 'r')
    ObjList = temp.read().split('\n')[:-2]
    temp.close
    
    # Process the spectra
    print('Beginning processing of all spectra for %s' % target)
    pool = multiprocessing.Pool(multiprocessing.cpu_count())
    temp = pool.map(process_deimos_spectra, range(len(ObjList)))
    temp = list(zip(*temp))
    ObjNumber, wavelength, spec, norm_spec, spec_err, dv, RA_Dec = temp
    print('Completed processing of all spectra')
    
    # Save processed spectra
    print('Saving all processed spectra for %s to %s\n\n\n\n\n' % (target,OutputFile))
    np.savez(OutputDir + OutputFile, obj=ObjNumber, wavelength=wavelength,
             spec=spec, norm_spec=norm_spec, spec_err=spec_err, dv=dv, RA_Dec=RA_Dec)
    

Restoring Wavelength Template
Calculating matrix of distances between wavelengths...
Restoring DEIMOS continuum pixels
Restoring telluric mask from Kirby+ 2008...
Restoring spectral template...
Prepping cross-correlation calculation...
Beginning reduction for pegdig26
Restoring list of pegdig26 objects
Beginning processing of all spectra for pegdig26
Processing spectra for object: spec1d.pegmsk.000.0000096.fits
Processing spectra for object: spec1d.pegmsk.004.serendip2.fits
Processing spectra for object: spec1d.pegmsk.005.serendip1.fits
Processing spectra for object: spec1d.pegmsk.002.0000107.fits
Processing spectra for object: spec1d.pegmsk.004.0010809.fits
Processing spectra for object: spec1d.pegmsk.006.0011877.fits
Processing spectra for object: spec1d.pegmsk.008.0010174.fits
Processing spectra for object: spec1d.pegmsk.009.0011559.fits
Restoring spectra #000.0000096
Restoring spectra #002.0000107
Processing spectra for object: spec1d.pegmsk.009.serendip2.fits
Restoring spectra #00

Finding radial velocity from CCF for spectra #007.serendip1
013.0011369 has Radial Velocity = -300
Shifting spectra 037.0010227 to rest frame
Shifting spectra 013.0011369 to rest frame
022.0010175 has Radial Velocity = -321
Normalizing spectra 031.serendip1
Normalizing spectra 037.0010227
Normalizing spectra 013.0011369
Calculating weights...
Shifting spectra 022.0010175 to rest frame
Calculating weights...
Calculating weights...
Normalizing spectra 022.0010175
030.serendip1 has Radial Velocity = -358
Calculating weights...
Shifting spectra 030.serendip1 to rest frame
Normalizing spectra 030.serendip1
Calculating weights...
006.0011877 has Radial Velocity = -300
Shifting spectra 006.0011877 to rest frame
Interpolating spectra #033.serendip1
Normalizing spectra 006.0011877
Applying telluric mask for spectra 033.serendip1
Finding radial velocity from CCF for spectra #033.serendip1
Calculating weights...
025.0010726 has Radial Velocity = -314
Shifting spectra 025.0010726 to rest frame
Nor

Continuum calculation complete!
Calculating continuum...
Continuum calculation complete!


/global/software/sl-7.x86_64/modules/langs/python/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:185: RuntimeWarning: invalid value encountered in true_divide


Processing spectra for object: spec1d.pegmsk.021.0011853.fits
Restoring spectra #021.0011853
Interpolating spectra #021.0011853
Applying telluric mask for spectra 021.0011853
Finding radial velocity from CCF for spectra #021.0011853
Continuum calculation complete!
021.0011853 has Radial Velocity = -314
Shifting spectra 021.0011853 to rest frame


...

Processing spectra for object: spec1d.wlmmsk.071.0000279.fits
Restoring spectra #071.0000279
Processing spectra for object: spec1d.wlmmsk.071.serendip1.fits
Restoring spectra #071.serendip1
Missing RA or Dec
Interpolating spectra #071.0000279
Applying telluric mask for spectra 071.0000279
Finding radial velocity from CCF for spectra #071.0000279
Calculating continuum...
071.0000279 has Radial Velocity = -500
Shifting spectra 071.0000279 to rest frame
Normalizing spectra 071.0000279
Calculating weights...
Processing spectra for object: spec1d.wlmmsk.072.0001618.fits
Restoring spectra #072.0001618
Missing RA or Dec
Continuum calculation complete!
Interpolating spectra #072.0001618
Applying telluric mask for spectra 072.0001618
Finding radial velocity from CCF for spectra #072.0001618
072.0001618 has Radial Velocity = -334
Shifting spectra 072.0001618 to rest frame
Normalizing spectra 072.0001618
Continuum calculation complete!
Calculating weights...
Processing spectra for object: spec1d.

/global/software/sl-7.x86_64/modules/langs/python/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:185: RuntimeWarning: invalid value encountered in true_divide


Continuum calculation complete!


/global/software/sl-7.x86_64/modules/langs/python/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:185: RuntimeWarning: invalid value encountered in true_divide


Interpolating spectra #077.serendip2
Applying telluric mask for spectra 077.serendip2
Finding radial velocity from CCF for spectra #077.serendip2
077.serendip2 has Radial Velocity = 250
Shifting spectra 077.serendip2 to rest frame
Normalizing spectra 077.serendip2
Calculating weights...
Continuum calculation complete!
Calculating continuum...
Calculating continuum...
Calculating continuum...
Calculating continuum...
Calculating continuum...
Continuum calculation complete!
Continuum calculation complete!
Calculating continuum...
Continuum calculation complete!
Continuum calculation complete!
Calculating continuum...
Calculating continuum...
Continuum calculation complete!
Calculating continuum...
Continuum calculation complete!
Calculating continuum...
Continuum calculation complete!
Continuum calculation complete!
Continuum calculation complete!
Continuum calculation complete!
Continuum calculation complete!
Calculating continuum...
Continuum calculation complete!
Calculating continuum